In [5]:
from services.pbiservice import PbiService
import os
from dotenv import load_dotenv
import json
from tinydb import TinyDB, Query


load_dotenv()
group_id = os.environ['GROUP_ID']
db = TinyDB('app_data.json')
dataset_table = db.table('dataset')

local_datasets = dataset_table.all()
dataset_is_configured = False

if len(local_datasets) > 0:
    pbi_dataset = PbiService().get_dataset_in_group_by_id(group_id, local_datasets[0])
    if pbi_dataset is None:
        dataset_table.truncate()
    else:
        dataset_is_configured = True
        dataset_id = local_datasets[0]['id']

if not dataset_is_configured:
    with open('dataset.json') as f:
        ds = json.load(f)
        res = PbiService().post_push_dataset_to_group(group_id, ds)
        dataset_table.insert(res.json())
        dataset_id = res.json()['id']
        











In [8]:
dataset_table.truncate()

In [33]:
import json
with open('dataset.json') as f:
    ds = json.load(f)

res = PbiService().post_push_dataset_to_group(group_id, ds)

In [1]:
import services.data_service as ds
df = ds.get_account_temperaments()

In [2]:
df['Open'].dtype

dtype('int64')

In [4]:
from utilities.data_utils import convert_dataframe_to_dict
convert_dataframe_to_dict(df)[0].get('Is Support Member')

True

# Main

In [8]:
from tinydb import TinyDB, Query
import json
import os
from dotenv import load_dotenv
import logging
import logging.config
import yaml
from services.pbiservice import PbiService
import services.data_service as ds
from utilities.data_utils import convert_dataframe_to_dict, get_table_columns

config = yaml.load(open("config.yml"), Loader=yaml.Loader)

logging.config.dictConfig(config['logging_config'])
logger = logging.getLogger('main')

load_dotenv()
group_id = os.environ['GROUP_ID']
db = TinyDB('app_data.json')
dataset_table = db.table('dataset')
local_datasets = dataset_table.all()
q = Query()

logger.info('Initiating data refresh process')

tables_to_push = []



dataset_is_configured = False

if len(local_datasets) > 0:
    pbi_dataset = PbiService().get_dataset_in_group_by_id(group_id, local_datasets[0])
    if pbi_dataset is None:
        dataset_table.truncate()
    else:
        dataset_is_configured = True
        dataset_id = local_datasets[0]['id']

if not dataset_is_configured:
    with open('dataset.json') as f:
        ds = json.load(f)
        res = PbiService().post_push_dataset_to_group(group_id, ds)
        dataset_table.insert(res.json())
        dataset_id = res.json()['id']

try: 
    logger.info('Fetching and processing case data from Salesforce')
    df_case_data = ds.get_case_data()
    df_case_data = df_case_data.head(5)
    df_case_data = df_case_data[get_table_columns('case_data')]
    case_data = convert_dataframe_to_dict(df_case_data)
    tables_to_push.append({'name': 'case_data', 'data': case_data})
except: 
    logger.exception('Something went wrong with fetching/processing case data')

try: 
    logger.info('Fetching and processing account temperament data from Salesforce')
    df_acct_temperaments = ds.get_account_temperaments(case_data=df_case_data)
    df_acct_temperaments = df_acct_temperaments[get_table_columns('account_temperaments')]
    acct_temperaments = convert_dataframe_to_dict(df_acct_temperaments)
    tables_to_push.append({'name': 'account_temperaments', 'data': acct_temperaments})
except: 
    logger.exception('Something went wrong with fetching/processing account temperament data')

try: 
    logger.info('Fetching and processing survey data from Salesforce')
    df_survey_data = ds.get_survey_data()
    df_survey_data = df_survey_data[get_table_columns('survey_data')]
    survey_data = convert_dataframe_to_dict(df_survey_data)
    tables_to_push.append({'name': 'survey_data', 'data': survey_data})
except: 
    logger.exception('Something went wrong with fetching/processing survey data')

try: 
    logger.info('Fetching birthday data')
    df_birthdays = ds.get_birthdays()
    df_birthdays = df_birthdays[get_table_columns('birthdays')]
    birthdays = convert_dataframe_to_dict(df_birthdays)
    tables_to_push.append({'name': 'birthdays', 'data': birthdays})
except: 
    logger.exception('Something went wrong with fetching birthday data')

try: 
    logger.info('Fetching datetime data')
    df_date_time = ds.get_time_as_dataframe()
    df_date_time = df_date_time[get_table_columns('date_time')]
    date_time = convert_dataframe_to_dict(df_date_time)
    tables_to_push.append({'name': 'date_time', 'data': date_time})
except: 
    logger.exception('Something went wrong with fetching datetime data')

try:
    logger.info('Posting table data to Power BI push dataset')
    pbiService = PbiService()
    for table in tables_to_push:
        pbiService.post_data_to_push_dataset_table(group_id, dataset_id, table.get('name'), table.get('data'))
except: 
    logger.exception('Something went wrong with posting table data to Power BI push dataset')



12-28-2023 00:07:38 - INFO - Initiating data refresh process
12-28-2023 00:07:38 - INFO - Fetching and processing case data from Salesforce


12-28-2023 00:07:49 - INFO - Fetching and processing account temperament data from Salesforce
12-28-2023 00:07:50 - INFO - Fetching and processing survey data from Salesforce
12-28-2023 00:07:52 - INFO - Fetching birthday data
12-28-2023 00:07:52 - INFO - Fetching datetime data
12-28-2023 00:07:52 - INFO - Posting table data to Power BI push dataset
Error while pushing data to datasetL
Not Found:	{"error":{"code":"ItemNotFound","message":"Dataset <pi>7e93764f-77dc-47a5-936b-f9f9b4d8cd72</pi> is not found."}}
RequestId:	b8ecdf88-d6ce-42e8-b228-c68f60ecc9f2
404 Client Error: Not Found for url: https://api.powerbi.com/v1.0/myorg/groups/6c2b4701-34e6-48d9-af12-d3528cda93d1/datasets/7e93764f-77dc-47a5-936b-f9f9b4d8cd72/tables/case_data/rows
Error while pushing data to datasetL
Not Found:	{"error":{"code":"ItemNotFound","message":"Dataset <pi>7e93764f-77dc-47a5-936b-f9f9b4d8cd72</pi> is not found."}}
RequestId:	020bde1a-b5fb-4053-851c-2f8dd670c4a6
404 Client Error: Not Found for url: https:/